In [18]:
import pandas as pd
import ast
import json
import os
from datetime import datetime

# Read the CSV file
df = pd.read_csv(r"C:\Users\User\Projects\Hyro\Senior data analyst assignment - Hyro.csv")

In [20]:
# saving up our conversation dimensions to the side
conversation_dim = df.drop(columns=['LIST_OF_EVENTS_TYPES_IN_ORDER','LIST_OF_EVENTS_DATA_IN_ORDER'])

In [21]:
# Convert the string representation of the list to an actual Python list
df['LIST_OF_EVENTS_TYPES_IN_ORDER'] = df['LIST_OF_EVENTS_TYPES_IN_ORDER'].apply(ast.literal_eval)

In [22]:
# Create a new DataFrame with both the event type and its position
List_of_events = df.apply(lambda row: 
    pd.DataFrame({
        'CONVERSATION_ID': row['CONVERSATION_ID'],
        'ASSISTANT_ID': row['ASSISTANT_ID'],
        'EVENT_TYPE': row['LIST_OF_EVENTS_TYPES_IN_ORDER'],
        'EVENT_POSITION': range(1, len(row['LIST_OF_EVENTS_TYPES_IN_ORDER']) + 1)
    })
, axis=1)

# Concatenate the individual DataFrames
List_of_events = pd.concat(List_of_events.tolist(), ignore_index=True)

# Display the first few rows to verify and check the number of rows
print(List_of_events.head())
print("\nNumber of rows:", len(List_of_events))

                        CONVERSATION_ID                          ASSISTANT_ID  \
0  6bbd6279-e3e8-4292-ad47-bc245df28c29  08fd0ad8-b8bd-4c6d-a1bf-c381dbe24b4d   
1  6bbd6279-e3e8-4292-ad47-bc245df28c29  08fd0ad8-b8bd-4c6d-a1bf-c381dbe24b4d   
2  6bbd6279-e3e8-4292-ad47-bc245df28c29  08fd0ad8-b8bd-4c6d-a1bf-c381dbe24b4d   
3  6bbd6279-e3e8-4292-ad47-bc245df28c29  08fd0ad8-b8bd-4c6d-a1bf-c381dbe24b4d   
4  6bbd6279-e3e8-4292-ad47-bc245df28c29  08fd0ad8-b8bd-4c6d-a1bf-c381dbe24b4d   

                 EVENT_TYPE  EVENT_POSITION  
0  CONVERSATION_TTL_SECONDS               1  
1          CONNECTION_EVENT               2  
2               WIDGET_LOAD               3  
3  CONVERSATION_TTL_SECONDS               4  
4          CONNECTION_EVENT               5  

Number of rows: 161419


In [23]:
# Try to parse the JSON without everything breaking
def safe_parse_json(json_str):
    try:
        # First, attempt to parse as JSON
        return json.loads(json_str)
    except json.JSONDecodeError:
        try:
            # If JSON parsing fails, try to clean and parse
            # Remove any Python-like list syntax
            cleaned_str = json_str.replace('\n', '').replace(' ', '')
            return json.loads(cleaned_str)
        except:
            # If all else fails, return the original string
            return json_str

# Apply the safe parsing function
df['LIST_OF_EVENTS_DATA_IN_ORDER'] = df['LIST_OF_EVENTS_DATA_IN_ORDER'].apply(safe_parse_json)

# Create a new DataFrame with event data info
List_of_events_data = df.apply(lambda row: 
    pd.DataFrame({
        'CONVERSATION_ID': row['CONVERSATION_ID'],
        'ASSISTANT_ID': row['ASSISTANT_ID'],
        'EVENT_POSITION': range(1, len(row['LIST_OF_EVENTS_DATA_IN_ORDER']) + 1),
        'EVENT_DATA': row['LIST_OF_EVENTS_DATA_IN_ORDER']
    })
, axis=1)

# Concatenate the individual DataFrames
List_of_events_data = pd.concat(List_of_events_data.tolist(), ignore_index=True)

# Display the first few rows to verify
print(List_of_events_data.head())

# Check the number of rows
print("\nNumber of rows:", len(List_of_events_data))

                        CONVERSATION_ID                          ASSISTANT_ID  \
0  6bbd6279-e3e8-4292-ad47-bc245df28c29  08fd0ad8-b8bd-4c6d-a1bf-c381dbe24b4d   
1  6bbd6279-e3e8-4292-ad47-bc245df28c29  08fd0ad8-b8bd-4c6d-a1bf-c381dbe24b4d   
2  6bbd6279-e3e8-4292-ad47-bc245df28c29  08fd0ad8-b8bd-4c6d-a1bf-c381dbe24b4d   
3  6bbd6279-e3e8-4292-ad47-bc245df28c29  08fd0ad8-b8bd-4c6d-a1bf-c381dbe24b4d   
4  6bbd6279-e3e8-4292-ad47-bc245df28c29  08fd0ad8-b8bd-4c6d-a1bf-c381dbe24b4d   

   EVENT_POSITION                                         EVENT_DATA  
0               1  {'eventData': {'ttl': 900}, 'eventId': '670ea9...  
1               2  {'eventData': {'isTest': False, 'location': {'...  
2               3  {'eventData': {'isTest': False, 'location': {'...  
3               4  {'eventData': {'ttl': 900}, 'eventId': '670ea9...  
4               5  {'eventData': {'isTest': False, 'location': {'...  

Number of rows: 161419


In [24]:
# Merge the two DataFrames
List_of_events_data = List_of_events_data.merge(
    List_of_events, 
    on=['CONVERSATION_ID', 'ASSISTANT_ID', 'EVENT_POSITION'], 
    how='left'
)

# Check the columns after merging
print("\nColumns after merge:", list(List_of_events_data.columns))


Columns after merge: ['CONVERSATION_ID', 'ASSISTANT_ID', 'EVENT_POSITION', 'EVENT_DATA', 'EVENT_TYPE']


In [25]:
# Let's get all unique event types
unique_event_types = List_of_events_data['EVENT_TYPE'].unique()

# Create a dictionary to store DataFrames for each event type
event_type_dfs = {}

# Iterate through unique event types
for event_type in unique_event_types:
    # Filter the DataFrame for the current event type
    df_filtered = List_of_events_data[List_of_events_data['EVENT_TYPE'] == event_type].copy()
    
    # Parse the EVENT_DATA for this specific type
    df_parsed = df_filtered.apply(lambda row: 
        pd.Series({
            'CONVERSATION_ID': row['CONVERSATION_ID'],
            'ASSISTANT_ID': row['ASSISTANT_ID'],
            'EVENT_POSITION': row['EVENT_POSITION'],
            **row['EVENT_DATA'],  # Unpack the main dictionary
            **row['EVENT_DATA'].get('eventData', {})  # Unpack nested eventData
        }), axis=1)
    
    # Store the parsed DataFrame in the dictionary
    event_type_dfs[event_type] = df_parsed

# Print out the event types and their DataFrame sizes
for event_type, df in event_type_dfs.items():
    print(f"Event Type: {event_type}")
    print(f"Number of rows: {len(df)}")
    print("Columns:", list(df.columns))
    print("\n")

Event Type: CONVERSATION_TTL_SECONDS
Number of rows: 16393
Columns: ['CONVERSATION_ID', 'ASSISTANT_ID', 'EVENT_POSITION', 'eventData', 'eventId', 'timestamp', 'typeOfEvent', 'ttl']


Event Type: CONNECTION_EVENT
Number of rows: 14118
Columns: ['CONVERSATION_ID', 'ASSISTANT_ID', 'EVENT_POSITION', 'eventData', 'eventId', 'timestamp', 'typeOfEvent', 'isTest', 'location']


Event Type: WIDGET_LOAD
Number of rows: 14118
Columns: ['CONVERSATION_ID', 'ASSISTANT_ID', 'EVENT_POSITION', 'eventData', 'eventId', 'timestamp', 'typeOfEvent', 'isTest', 'location', 'referrer']


Event Type: WIDGET_CLICK
Number of rows: 9717
Columns: ['CONVERSATION_ID', 'ASSISTANT_ID', 'EVENT_POSITION', 'eventData', 'eventId', 'timestamp', 'typeOfEvent', 'name']


Event Type: SWITCH_COMMAND
Number of rows: 4880
Columns: ['ASSISTANT_ID', 'CONVERSATION_ID', 'EVENT_POSITION', 'commandType', 'data', 'eventData', 'eventId', 'sender', 'timestamp', 'typeOfEvent']


Event Type: THINKING
Number of rows: 51687
Columns: ['CONVERS

In [26]:
# Function to convert unix timestamp (milliseconds) to datetime
def convert_unix_to_datetime(unix_ms):
    try:
        return datetime.fromtimestamp(unix_ms / 1000.0)
    except:
        return None

# Iterate through each DataFrame in event_type_dfs
for event_type, df in event_type_dfs.items():
    # Check if 'timestamp' column exists in this DataFrame
    if 'timestamp' in df.columns:
        # Convert the timestamp
        df['timestamp'] = df['timestamp'].apply(convert_unix_to_datetime)
        print(f"Converted timestamps for: {event_type}")

# Verify the conversion for each DataFrame
for event_type, df in event_type_dfs.items():
    if 'timestamp' in df.columns:
        print(f"\nEvent Type: {event_type}")
        print("Timestamp sample:")
        print(df['timestamp'].head())

Converted timestamps for: CONVERSATION_TTL_SECONDS
Converted timestamps for: CONNECTION_EVENT
Converted timestamps for: WIDGET_LOAD
Converted timestamps for: WIDGET_CLICK
Converted timestamps for: SWITCH_COMMAND
Converted timestamps for: THINKING
Converted timestamps for: BOT_UTTERANCE
Converted timestamps for: SUGGESTION_CLICK
Converted timestamps for: USER_UTTERANCE
Converted timestamps for: MAPPED_TERM
Converted timestamps for: KNOWLEDGE
Converted timestamps for: FOLLOWUP_QUESTION
Converted timestamps for: SAID
Converted timestamps for: PROVIDER_MEDICAL_ATTRIBUTE
Converted timestamps for: SEARCH_PERFORMED
Converted timestamps for: BOT_SOCKET_CLOSED
Converted timestamps for: CONVERSATION_TTL_REACHED
Converted timestamps for: INTENT
Converted timestamps for: CANNED_RESPONSE
Converted timestamps for: MISUNDERSTANDING
Converted timestamps for: URL_TAG_CLICK
Converted timestamps for: CHAT_CONVERSATION_ENDED
Converted timestamps for: RESULT_CLICK
Converted timestamps for: CTA_CLICK
Conver

In [27]:
# Creating a directory for all of our exported data
export_dir = os.path.join(os.getcwd(), 'data_tables')
os.makedirs(export_dir, exist_ok=True)

# Export each event type DataFrame
for event_type, df in event_type_dfs.items():
    # Create a filename by replacing any spaces or special characters with underscores
    filename = f"{event_type.replace(' ', '_').replace('/', '_')}_events.csv"
    filepath = os.path.join(export_dir, filename)
    
    # Export to CSV
    df.to_csv(filepath, index=False)
    print(f"Exported: {filename}")

# Export our main List_of_events_data
list_events_data_filepath = os.path.join(export_dir, 'List_of_events_data.csv')
List_of_events_data.to_csv(list_events_data_filepath, index=False)
print(f"\nExported: List_of_events_data.csv")

# Export our conversation dimension 
conversation_dim_filepath = os.path.join(export_dir, 'Conversation_dimension.csv')
conversation_dim.to_csv(conversation_dim_filepath, index=False)
print(f"\nExported: Conversation_dimension.csv")

# Print the directory where files are saved
print(f"\nAll files exported to: {export_dir}")

Exported: CONVERSATION_TTL_SECONDS_events.csv
Exported: CONNECTION_EVENT_events.csv
Exported: WIDGET_LOAD_events.csv
Exported: WIDGET_CLICK_events.csv
Exported: SWITCH_COMMAND_events.csv
Exported: THINKING_events.csv
Exported: BOT_UTTERANCE_events.csv
Exported: SUGGESTION_CLICK_events.csv
Exported: USER_UTTERANCE_events.csv
Exported: MAPPED_TERM_events.csv
Exported: KNOWLEDGE_events.csv
Exported: FOLLOWUP_QUESTION_events.csv
Exported: SAID_events.csv
Exported: PROVIDER_MEDICAL_ATTRIBUTE_events.csv
Exported: SEARCH_PERFORMED_events.csv
Exported: BOT_SOCKET_CLOSED_events.csv
Exported: CONVERSATION_TTL_REACHED_events.csv
Exported: INTENT_events.csv
Exported: CANNED_RESPONSE_events.csv
Exported: MISUNDERSTANDING_events.csv
Exported: URL_TAG_CLICK_events.csv
Exported: CHAT_CONVERSATION_ENDED_events.csv
Exported: RESULT_CLICK_events.csv
Exported: CTA_CLICK_events.csv
Exported: MAP_CLICK_events.csv
Exported: FEEDBACK_events.csv
Exported: START_RECORDING_events.csv
Exported: AUDIO_STREAM_event